In [1]:
from functools import partial

# import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# Plotly looks sharper!
import plotly.graph_objects as go

from intransparent import (
    YEAR_LABELS,
    ingest_reports_per_country,
    without_populations,

    REPORTS_PER_PLATFORM,
    ingest_reports_per_platform,
    compare_all_platform_reports,

    show_html,
    show_info,
    show_table,
)

show_html(h1='CSAM Reports per Country', h2='Prelude')

logger = partial(show_info, invocation='logger(')
tables = ingest_reports_per_country('./data', logger=logger)

countries_without, reports_without = (
    without_populations(tables.reports, tables.populations))
show_html(
    f'{countries_without.shape[0]} countries with reports have no population '
    'statistics. They also account for very few reports.')
show_table(reports_without)


,reports,reports_pct
year,,
2019,28,0.000165
2020,97,0.000446
2021,243,0.000827
2022,117,0.000365


In [2]:
show_html(
    h2='Regions with Most CSAM Reports', 
    p='Selection does <em>not</em> take population counts into consideration.')

most_reports = (
    tables.reports_per_country
    .groupby(['year', 'region'])
    .sum(numeric_only=True)
    .sort_values(by='reports', ascending=False)
)

for year in YEAR_LABELS:
    show_table(
        most_reports.query(f'year == "{year}"').head(20),
        table_name=f'Regions with most CSAM Reports {year}',
        highlight=['reports', 'reports_pct'],
    )


In [3]:
show_html(h2='Countries with Most CSAM Reports Per Capita')

per_capita = (
    tables.reports_per_country
    .sort_values(by='reports_per_capita', ascending=False)
    .groupby('year') 
)

for year, group in per_capita:
    group = group.head(20).reset_index()
    show_table(
        group,
        table_name=f'reports per capita {year}',
        highlight='reports_per_capita',
    )

,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country,region,superregion,continent
0,ARE,2019,"330,268",1.944198,"9,211,657",0.118631,0.035853,United Arab Emirates,Western Asia,Asia,Asia
1,IRQ,2019,"1,026,809",6.044547,"41,563,521",0.535271,0.024705,Iraq,Western Asia,Asia,Asia
2,LBY,2019,"150,848",0.888001,"6,569,088",0.084599,0.022963,Libya,Northern Africa,Africa,Africa
3,QAT,2019,"53,467",0.314746,"2,807,235",0.036153,0.019046,Qatar,Western Asia,Asia,Asia
4,BHR,2019,"26,093",0.153602,"1,494,189",0.019243,0.017463,Bahrain,Western Asia,Asia,Asia
5,DZA,2019,"700,535",4.123860,"42,705,369",0.549976,0.016404,Algeria,Northern Africa,Africa,Africa
6,SYC,2019,"1,501",0.008836,"104,373",0.001344,0.014381,Seychelles,Eastern Africa,Sub-Saharan Africa,Africa
7,SAU,2019,"514,832",3.030677,"35,827,362",0.461398,0.014370,Saudi Arabia,Western Asia,Asia,Asia
8,KWT,2019,"63,492",0.373760,"4,441,100",0.057194,0.014296,Kuwait,Western Asia,Asia,Asia
9,JOR,2019,"123,537",0.727229,"10,698,684",0.137782,0.011547,Jordan,Western Asia,Asia,Asia


,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country,region,superregion,continent
0,LBY,2020,"189,955",0.873313,"6,653,942",0.084861,0.028548,Libya,Northern Africa,Africa,Africa
1,DZA,2020,"1,102,939",5.070731,"43,451,666",0.554163,0.025383,Algeria,Northern Africa,Africa,Africa
2,MCO,2020,886,0.004073,"36,922",0.000471,0.023997,Monaco,Western Europe,Europe,Europe
3,ARE,2020,"216,874",0.997072,"9,287,289",0.118446,0.023352,United Arab Emirates,Western Asia,Asia,Asia
4,IRQ,2020,"919,966",4.229518,"42,556,984",0.542753,0.021617,Iraq,Western Asia,Asia,Asia
5,QAT,2020,"56,683",0.260598,"2,760,385",0.035205,0.020534,Qatar,Western Asia,Asia,Asia
6,BTN,2020,"15,379",0.070705,"772,506",0.009852,0.019908,Bhutan,Southern Asia,Asia,Asia
7,MAR,2020,"705,863",3.245185,"36,688,772",0.467912,0.019239,Morocco,Northern Africa,Africa,Africa
8,KWT,2020,"68,582",0.315304,"4,360,445",0.055611,0.015728,Kuwait,Western Asia,Asia,Asia
9,BHR,2020,"22,960",0.105558,"1,477,469",0.018843,0.015540,Bahrain,Western Asia,Asia,Asia


,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country,region,superregion,continent
0,LBY,2021,"270,811",0.921199,"6,735,277",0.085156,0.040208,Libya,Northern Africa,Africa,Africa
1,ARE,2021,"327,820",1.115122,"9,365,145",0.118407,0.035004,United Arab Emirates,Western Asia,Asia,Asia
2,IRQ,2021,"1,220,470",4.151586,"43,533,593",0.550411,0.028035,Iraq,Western Asia,Asia,Asia
3,PHL,2021,"3,188,793",10.847090,"113,880,328",1.439829,0.028001,Philippines,South-Eastern Asia,Asia,Asia
4,QAT,2021,"73,536",0.250142,"2,688,235",0.033988,0.027355,Qatar,Western Asia,Asia,Asia
5,DZA,2021,"1,171,653",3.985529,"44,177,969",0.558558,0.026521,Algeria,Northern Africa,Africa,Africa
6,KHM,2021,"395,793",1.346341,"16,589,024",0.209741,0.023859,Cambodia,South-Eastern Asia,Asia,Asia
7,BHR,2021,"32,346",0.110029,"1,463,266",0.018501,0.022105,Bahrain,Western Asia,Asia,Asia
8,BLZ,2021,"8,297",0.028223,"400,031",0.005058,0.020741,Belize,Central America,Latin America and the Caribbean,Americas
9,SAU,2021,"708,980",2.411687,"35,950,396",0.454534,0.019721,Saudi Arabia,Western Asia,Asia,Asia


,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country,region,superregion,continent
0,ARE,2022,"265,307",0.827558,"9,441,129",0.118383,0.028101,United Arab Emirates,Western Asia,Asia,Asia
1,LBY,2022,"163,336",0.509485,"6,812,341",0.085420,0.023976,Libya,Northern Africa,Africa,Africa
2,QAT,2022,"61,167",0.190795,"2,695,122",0.033794,0.022695,Qatar,Western Asia,Asia,Asia
3,PHL,2022,"2,576,182",8.035746,"115,559,009",1.448997,0.022293,Philippines,South-Eastern Asia,Asia,Asia
4,IRQ,2022,"905,883",2.825672,"44,496,122",0.557938,0.020359,Iraq,Western Asia,Asia,Asia
5,BHR,2022,"25,896",0.080776,"1,472,233",0.018460,0.017590,Bahrain,Western Asia,Asia,Asia
6,SAU,2022,"602,745",1.880110,"36,408,820",0.456531,0.016555,Saudi Arabia,Western Asia,Asia,Asia
7,DZA,2022,"731,167",2.280690,"44,903,225",0.563042,0.016283,Algeria,Northern Africa,Africa,Africa
8,BLZ,2022,"6,103",0.019037,"405,273",0.005082,0.015059,Belize,Central America,Latin America and the Caribbean,Americas
9,MDV,2022,"7,813",0.024371,"523,787",0.006568,0.014916,Maldives,Southern Asia,Asia,Asia


In [40]:
for year in YEAR_LABELS:
    years_worth = tables.reports_per_country.query(f'year == "{year}"')
    map_data = tables.geometries.merge(years_worth, left_on='iso3', right_on='iso3')
    map_data = map_data.reset_index()
    show_info(map_data)

    fig = go.Figure(
        data=go.Choropleth(
            locations=map_data['iso3'],
            locationmode='ISO-3',
            z=map_data['reports_per_capita'],
            text=map_data['country'],
            colorscale='Plasma_r',
            showscale=True,
            colorbar_title='Reports<br>per Capita',
        ),
        layout=dict(
            width=600,
            height=300,
            margin={"r":0,"t": 40, "l":0,"b":0},
            geo=dict(
                showframe=False,
                lataxis_range=[-60,90], # Crop antarctica out of map
                bgcolor='#eaeaec',
            ),
            #paper_bgcolor='#000000', # Enable when rightsizing the height
        ),
    )

    fig.update_traces(
        marker_line_width=0.4,
        colorbar_thickness=20,
        colorbar_tickformat='.3f',
        colorbar_title_font_size=13,
        selector=dict(type='choropleth'),
    ) 
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
        title_text=f'CSAM Reports per Capita {year}',
    )
    fig.show()


In [4]:
show_html(h2='CSAM Reports per Social Media Platform')

disclosures = ingest_reports_per_platform(
    REPORTS_PER_PLATFORM, include_redundant=True)
comparisons = compare_all_platform_reports(disclosures)
for platform, data in comparisons.items():
    show_table(data, table_name=platform)

,pieces,reports,Δ%,NCMEC
2020,,"3,432",0.000000,"3,432"
2021,"1,608","2,684",-14.940387,"2,283"
2022,"5,530","4,969",590.480982,"34,310"


,pieces,reports,Δ%,NCMEC
2019,,724,0.000000,724
2020,,"2,233",0.000000,"2,233"
2021,"9,258","10,059",0.000000,"10,059"
2022,"80,888","52,592",0.000000,"52,592"


,pieces,reports,Δ%,NCMEC
2019,,,,"82,030"
2020,,,,"144,095"
2021,,,,"512,522"
2022,"746,051","285,470",93.045154,"551,086"


,pieces,reports,Δ%,NCMEC
2020,"4,437,853","547,875",-0.213735,"546,704"
2021,"6,696,497","870,319",0.627816,"875,783"
2022,"13,402,885","2,174,319",0.010532,"2,174,548"
